In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes
import py_vollib_vectorized

In [2]:
def scale_gamma(arr, spot):
    return arr*spot*spot/100
    

In [3]:
def create_adjusted_texp_grid(texp_grid, option_texp, max_texp):
    texp_gridi=texp_grid-(max_texp-option_texp)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    return texp_gridi_zero

def create_constant_grid(param, num_time_steps, num_spots):
    cp_grid=np.full((num_time_steps,num_spots),param, dtype=type(param))
    return cp_grid
#create_constant_grid(35.0, num_time_steps, num_spots)


In [5]:
import ipysheet
sheet = ipysheet.sheet(rows=3, columns=4)
cell1 = ipysheet.cell(0, 0, 'Hello')
cell2 = ipysheet.cell(2, 0, 'World')
cell_value = ipysheet.cell(2,2, 42.)

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Hello'), Cell(colu…

Create option parameters from arrays and put them into a dataframe


In [64]:
qty=np.array([-1,1]*2)*1000000
texp=[1,1,2,2]
k=[.9,1.1]*2
k_dlr=k
cp=['p','c']*2
sigma=[.20,.2]*2

num_time_steps=10
num_spots=19
low_strike=.5
incr_strike=.1
high_strike=2.0
max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
#spot_dim=np.linspace(low_strike,high_strike,num_spots)
spot_dim=np.arange(low_strike,high_strike,incr_strike)
num_spots=len(spot_dim)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)



In [51]:
data = {'qty': qty,
        'texp': texp,
        'k':k,
        '$k':k,
        'cp': cp,
        'sigma':sigma}
df_inp=pd.DataFrame(data)

In [79]:
spot=100
r=0
q=0
def price_input_df(r, q):
    option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(df_inp['cp'], 1, df_inp['k'],df_inp['texp'], r, df_inp['sigma'],q,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(df_inp['cp'], 1, df_inp['k'], df_inp['texp'], r, df_inp["sigma"],q,model='black_scholes_merton', return_as='dict')
    df_inp['cp']
    df_inp["price"]=option_price
    df_inp["delta"]=option_greeks["delta"]
    df_inp["gamma"]=option_greeks["gamma"]
    df_inp["vega"]=option_greeks["vega"]

price_input_df(r, q)
df_inp


,qty,texp,k,$k,cp,sigma,price,delta,gamma,vega
0,-1000000,1,0.9,0.9,p,0.2,0.035891,-0.265507,1.638903,0.003277
1,1000000,1,1.1,1.1,c,0.2,0.042920,0.353281,1.857804,0.003715
2,-1000000,2,0.9,0.9,p,0.2,0.064111,-0.303710,1.235963,0.004943
3,1000000,2,1.1,1.1,c,0.2,0.074677,0.422474,1.383622,0.005534


In [80]:
def price_df_one_row(row):
    cp=row["cp"][0,0]
    spot=row["spot"]
    texp=row["texp"]
    r=row["r"]
    sigma=row["sigma"]
    k=row["k"]
    q=row["q"]
    spot_jump=spot*(1-.25)
    option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(cp, spot, k, texp, r, sigma,q,return_as='array')
    option_price_jump=py_vollib_vectorized.models.vectorized_black_scholes_merton(cp, spot_jump, k, texp, r, sigma,q,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(cp, spot, k, texp, r, sigma,q,model='black_scholes_merton', return_as='dict')
    option_greeks["price"]=option_price
    option_price_jump=np.array(option_price_jump).reshape(spot.shape)
    option_greeks = {k: np.array(v).reshape(spot.shape) for k, v in option_greeks.items()}
    option_greeks["jump_pl"]=(option_price_jump-option_greeks["price"])-option_greeks["delta"]*(spot_jump-spot)
    option_greeks = {k: np.where(texp>0,v,0) for k, v in option_greeks.items()}
    
    return(option_greeks)
    
def create_input_grids(r, q,spot_price, df_inp):
    df_grids=df_inp.copy()

    df_grids["r"]=r
    df_grids["q"]=q
    df_grids["texp"]=df_grids["texp"].apply(lambda x: create_adjusted_texp_grid(texp_grid, x, max_texp))
    df_grids[["qty","k","cp","sigma"]]=df_inp[["qty","k","cp","sigma"]].applymap(lambda x: create_constant_grid(x, num_time_steps, num_spots))
    df_grids[["q","r"]]=df_grids[["q","r"]].applymap(lambda x: create_constant_grid(x, num_time_steps, num_spots))
    df_grids[["spot"]]=df_inp[["qty"]].applymap(lambda x: spot_grid)
    return df_grids

def stack_bs_outputs(df_grids, df_priced):
    dict_grids={}
    for field in df_priced[0].keys():
        dict_grids[field]=np.stack([el[field] for el in df_priced.values()])

    dict_grids["qty"]=np.stack(list(df_grids["qty"]))
    dict_grids["texp"]=np.stack(list(df_grids["texp"]))
    dict_grids["gamma"]*=spot_grid/100
    dict_sum_grids={}
    for field in ["price","delta","gamma","vega","rho","theta","jump_pl"]:
        dict_sum_grids[field]=(dict_grids[field]*dict_grids["qty"]).sum(axis=0)
    return (dict_grids, dict_sum_grids)

def run_pricing(r, q, spot_price,df_inp):
    df_grids = create_input_grids(r, q,spot_price,df_inp)
    df_priced={ind: price_df_one_row(row) for (ind,row) in df_grids.iterrows()}
    (dict_grids, dict_sum_grids) = stack_bs_outputs(df_grids, df_priced)
    return (dict_grids, dict_sum_grids)

def price_input_df(r, q):
    option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(df_inp['cp'], 1, df_inp['k'],df_inp['texp'], r, df_inp['sigma'],q,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(df_inp['cp'], 1, df_inp['k'], df_inp['texp'], r, df_inp["sigma"],q,model='black_scholes_merton', return_as='dict')
    df_inp['cp']
    df_inp["price"]=option_price
    df_inp["delta"]=option_greeks["delta"]
    df_inp["gamma"]=option_greeks["gamma"]
    df_inp["vega"]=option_greeks["vega"]



2.0

In [53]:
r=0
q=0
spot_price=100
(dict_grids, dict_sum_grids)=run_pricing(r, q, spot_price, df_inp)


In [97]:
import ipywidgets as widgets
from IPython.display import display
scales_factor={'price':1,'delta':1,'gamma':1/1000000*spot_price,'vega':1/1000*spot_price,'rho':spot_price,'theta':1,'jump_pl':spot_price/1000000}
show_fields=["price","delta","gamma","vega","rho","theta","jump_pl"]

w_greek_selector=widgets.Dropdown(
    options=list(dict_sum_grids.keys()),
    value=list(dict_sum_grids.keys())[3],
    #rows=10,
    description='Columns to graph',
    disabled=False
)
w_min_spot=widgets.FloatRangeSlider(
    value=[5,200],
    min=0.0,
    max=1000.0,
    step=0.5,
    description='Spot:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='0.1f'
)
w_time_steps=widgets.BoundedIntText(
    value=10,
    min=1,
    max=20.0,
    step=1,
    description='Time Step:',
    disabled=False
)

w_spot=widgets.BoundedFloatText(
    value=100,
    min=0,
    max=100000,
    step=1,
    description='Spot:',
    disabled=False
)
w_Div=widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=.005,
    description='Div:',
    disabled=False
)
w_IR=widgets.BoundedFloatText(
    value=000,
    min=.000,
    max=100000,
    step=1,
    description='IR:',
    disabled=False
)

w_outputInputPricing=widgets.Output()
w_layout_box=widgets.HBox([widgets.VBox([w_greek_selector,w_spot,w_Div,w_IR]),w_outputInputPricing])

def display_field(field, spot, div, ir):
    (dict_grids, dict_sum_grids)=run_pricing(ir/100, div/100, spot, df_inp)
    scales_factor={'price':1,'delta':1,'gamma':1/1000000*spot,'vega':1/1000*spot,'rho':spot,'theta':1,'jump_pl':spot/1000000}
    df = pd.DataFrame(dict_sum_grids[field]*scales_factor[field], columns = spot_dim, index = texp_dim)
    display(df.style.format('{:.1f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2))
    price_input_df(r, q)
    df_inp["price"]*=spot
    df_inp["$k"]=df_inp["k"]*spot
    w_outputInputPricing.clear_output()
    with w_outputInputPricing:
        
        display(df_inp.style.format({"qty": "{:,d}", 
                          "texp": "{:.2f}", 
                          "price": "{:.2f}",
                          "k": "{:.2f}",
                          "$k": "${:.2f}",
                          "sigma": "{:.2%}", 
                          "delta":"{:.2%}"}))
out1 = widgets.interactive_output(display_field, {'field': w_greek_selector, 'spot': w_spot, 'div': w_Div, 'ir': w_IR})

display(w_layout_box)
display(out1)

Output()

In [15]:
dict_grids["delta"][3,:,:].shape
df = pd.DataFrame(dict_grids["price"][0,:,:], columns = spot_dim, index = texp_dim)
display(df.style.format('{:.3f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2))

,50.0%,57.5%,65.0%,72.5%,80.0%,87.5%,95.0%,102.5%,110.0%,117.5%,125.0%,132.5%,140.0%,147.5%,155.0%,162.5%,170.0%,177.5%,185.0%,192.5%,200.0%
2.00,0.400,0.326,0.253,0.186,0.129,0.084,0.051,0.030,0.016,0.009,0.004,0.002,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.80,0.400,0.325,0.252,0.183,0.123,0.077,0.044,0.023,0.012,0.005,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.60,0.400,0.325,0.251,0.180,0.117,0.068,0.036,0.017,0.007,0.003,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.40,0.400,0.325,0.250,0.177,0.110,0.058,0.026,0.010,0.003,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.20,0.400,0.325,0.250,0.175,0.103,0.046,0.014,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.80,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.60,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.40,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.20,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [20]:
np.arange(.05,2,.05)

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05, 1.1 ,
       1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 , 1.65,
       1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95])

- item 1 *italics* $$a_b*2=c$$
- item 2

1. item 1 
2. item 2